In [108]:
import numpy as np
import pandas as pd
import torch
from sklearn.neighbors import NearestNeighbors

In [109]:
#from Utils import train_knn, get_dist_idx, get_recommend_url
import pandas as pd
import time
from ast import literal_eval

# Suppressing Bert Warning
from transformers import logging
logging.set_verbosity_error()

file_path = '/Users/shuchenz/Desktop/Ops Con/ChangingRoom/Coding/CR-Vectorization/Vectorization/vectorized_987.csv'

In [110]:
from VecUtils import ImgVectorize, TextVectorize
import pandas as pd
df = pd.read_csv(file_path, converters= {'image_links':literal_eval})
df['ImgVec'] = df['ImgVec'].apply(lambda x: 
                           np.fromstring(
                               x.replace('\n','')
                                .replace('[','')
                                .replace(']','')
                                .replace('  ',' '), sep=' '))
df['TxtVec'] = torch.load('txtvec.pt').values
df['TxtVecSep'] = torch.load('txtvecsep.pt').values
for i in range(len(df)):
    df['TxtVec'][i] = list(df['TxtVec'][i])
    df['TxtVecSep'][i] = list(df['TxtVecSep'][i])

/var/folders/j3/njl9qzcj0lbb505pgvr7gsrh0000gn/T/ipykernel_1292/686074971.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['TxtVec'][i] = list(df['TxtVec'][i])
/var/folders/j3/njl9qzcj0lbb505pgvr7gsrh0000gn/T/ipykernel_1292/686074971.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['TxtVecSep'][i] = list(df['TxtVecSep'][i])


## Preprocessing

### Resnet

In [111]:
import numpy as np
import pandas as pd

array_resnet = []
check = 0
for i in df['ImgVec']:
    array_resnet.append(i) 

df_resnet = pd.DataFrame(array_resnet)

df_resnet

,0,1,2,3,4,5,6,7,8,9,...,502,503,504,505,506,507,508,509,510,511
0,0.821871,0.933361,0.920547,0.893624,0.828154,0.966634,0.923863,0.925982,0.881806,1.030577,...,0.812287,0.881183,0.948156,0.919897,0.882860,0.880982,1.038170,1.106688,0.955504,0.985731
1,0.973653,0.989243,0.945960,1.021952,0.841379,0.963154,0.870935,0.817570,0.991387,1.050257,...,1.070790,0.784580,1.125471,0.931358,0.936564,0.882438,1.009147,0.978824,0.918205,0.957741
2,1.000431,0.914692,1.031360,0.905025,1.038498,0.961392,0.973464,0.858744,1.051390,1.014303,...,1.019137,0.888430,0.978145,1.058011,0.916643,0.922747,0.999189,0.947187,0.799099,0.992933
3,0.876284,0.950117,1.141460,0.924114,0.902459,1.074697,0.870607,0.944230,1.087493,1.036616,...,0.968110,0.935491,0.794646,0.903564,0.839001,0.880484,1.105116,0.961103,0.838815,0.975532
4,1.083690,0.952222,0.909902,0.814844,0.845148,0.905851,0.912053,0.809988,1.070131,0.993851,...,0.825885,0.905046,0.946886,0.826032,0.852205,0.802403,1.050662,1.012539,0.830958,0.912152
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
982,0.875689,0.942713,1.061662,0.963320,0.936216,1.092020,1.007864,0.885109,1.234413,0.904729,...,0.854150,1.010205,0.932435,1.168394,0.869543,0.860036,0.971735,0.994484,0.914461,0.990262
983,0.880198,0.999473,1.111135,0.889674,1.029831,1.014353,1.019814,0.969593,1.051707,1.080274,...,0.791126,1.076153,1.106489,1.238878,0.812652,0.831462,0.977202,0.941709,0.859284,0.877920
984,0.919378,0.966359,0.979112,1.018469,0.967341,0.935031,0.967335,0.917793,1.138127,1.061330,...,0.934004,0.916723,0.807466,1.080032,0.809180,0.896269,1.093341,0.925228,0.856727,1.038363
985,0.888521,0.871960,1.085654,1.005369,0.912420,0.958169,0.833186,0.892758,0.929759,1.157060,...,0.899997,0.917389,0.966094,0.992391,0.735229,0.832600,0.999529,1.018686,0.905117,1.072215


### BERT

In [112]:
array_bert = []
for i in df['TxtVec']:
    array_bert.append(i) 
df_bert = pd.DataFrame(array_bert)
df_bert


,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
0,tensor(-0.8227),tensor(0.1953),tensor(0.1065),tensor(0.4472),tensor(-0.0157),tensor(0.7313),tensor(-0.2648),tensor(0.0243),tensor(0.2500),tensor(-1.2267),...,tensor(0.2064),tensor(-0.8635),tensor(-0.4525),tensor(0.1010),tensor(-0.1073),tensor(-0.7141),tensor(0.7107),tensor(0.3938),tensor(-0.1198),tensor(-0.0886)
1,tensor(-0.8803),tensor(0.0845),tensor(0.1897),tensor(0.1384),tensor(0.1310),tensor(0.8866),tensor(-0.0474),tensor(0.0882),tensor(0.1416),tensor(-1.1609),...,tensor(0.1252),tensor(-0.7908),tensor(-0.2941),tensor(-0.0235),tensor(-0.2452),tensor(-0.5948),tensor(0.6747),tensor(0.1880),tensor(-0.3803),tensor(-0.3122)
2,tensor(-0.9368),tensor(0.1136),tensor(0.1412),tensor(0.5675),tensor(-0.0745),tensor(0.6837),tensor(-0.2712),tensor(0.0167),tensor(0.3441),tensor(-1.3416),...,tensor(0.3198),tensor(-1.0450),tensor(-0.3971),tensor(0.0506),tensor(-0.1429),tensor(-0.7402),tensor(0.7716),tensor(0.4137),tensor(-0.0444),tensor(-0.1238)
3,tensor(-0.7935),tensor(0.2577),tensor(0.0124),tensor(0.4548),tensor(-0.0377),tensor(0.6651),tensor(-0.4552),tensor(0.0178),tensor(0.3869),tensor(-1.2125),...,tensor(0.3053),tensor(-0.9287),tensor(-0.4616),tensor(-0.1855),tensor(0.1151),tensor(-0.8613),tensor(0.7477),tensor(0.3395),tensor(-0.1691),tensor(0.0748)
4,tensor(-0.9021),tensor(0.2014),tensor(0.1325),tensor(0.5247),tensor(-0.0419),tensor(0.7302),tensor(-0.3188),tensor(0.0150),tensor(0.2526),tensor(-1.3101),...,tensor(0.2623),tensor(-0.9203),tensor(-0.4519),tensor(0.0827),tensor(-0.0945),tensor(-0.7653),tensor(0.7386),tensor(0.4356),tensor(-0.0860),tensor(-0.0833)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
982,tensor(-0.8005),tensor(0.1791),tensor(0.0880),tensor(0.3498),tensor(-0.0969),tensor(0.6585),tensor(-0.4464),tensor(0.0343),tensor(0.3995),tensor(-1.2354),...,tensor(0.2870),tensor(-0.8791),tensor(-0.4357),tensor(-0.0134),tensor(0.0495),tensor(-0.7856),tensor(0.8730),tensor(0.3396),tensor(-0.2642),tensor(0.0362)
983,tensor(-0.9018),tensor(0.1228),tensor(0.1406),tensor(0.5405),tensor(-0.0378),tensor(0.7090),tensor(-0.2819),tensor(0.0047),tensor(0.3253),tensor(-1.3306),...,tensor(0.2964),tensor(-1.0083),tensor(-0.4304),tensor(0.0434),tensor(-0.1411),tensor(-0.7327),tensor(0.7527),tensor(0.4104),tensor(-0.0835),tensor(-0.1544)
984,tensor(-0.9018),tensor(0.1697),tensor(0.1298),tensor(0.4747),tensor(-0.0141),tensor(0.7211),tensor(-0.2761),tensor(0.0358),tensor(0.3112),tensor(-1.3029),...,tensor(0.2496),tensor(-0.9465),tensor(-0.4339),tensor(0.0731),tensor(-0.1033),tensor(-0.7130),tensor(0.7284),tensor(0.3568),tensor(-0.1389),tensor(-0.0972)
985,tensor(-0.7620),tensor(0.2155),tensor(0.0365),tensor(0.0951),tensor(0.1594),tensor(1.0556),tensor(-0.1536),tensor(0.0429),tensor(0.0747),tensor(-1.1855),...,tensor(0.1972),tensor(-0.6311),tensor(-0.4613),tensor(0.0107),tensor(-0.0952),tensor(-0.7108),tensor(0.6025),tensor(0.1582),tensor(-0.3760),tensor(-0.1260)


### Resnet & BERT

In [113]:
frames = [df_resnet, df_bert]
df_bert_resnet = pd.concat(frames, axis=1)

df_bert_resnet

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
0,0.821871,0.933361,0.920547,0.893624,0.828154,0.966634,0.923863,0.925982,0.881806,1.030577,...,tensor(0.2064),tensor(-0.8635),tensor(-0.4525),tensor(0.1010),tensor(-0.1073),tensor(-0.7141),tensor(0.7107),tensor(0.3938),tensor(-0.1198),tensor(-0.0886)
1,0.973653,0.989243,0.945960,1.021952,0.841379,0.963154,0.870935,0.817570,0.991387,1.050257,...,tensor(0.1252),tensor(-0.7908),tensor(-0.2941),tensor(-0.0235),tensor(-0.2452),tensor(-0.5948),tensor(0.6747),tensor(0.1880),tensor(-0.3803),tensor(-0.3122)
2,1.000431,0.914692,1.031360,0.905025,1.038498,0.961392,0.973464,0.858744,1.051390,1.014303,...,tensor(0.3198),tensor(-1.0450),tensor(-0.3971),tensor(0.0506),tensor(-0.1429),tensor(-0.7402),tensor(0.7716),tensor(0.4137),tensor(-0.0444),tensor(-0.1238)
3,0.876284,0.950117,1.141460,0.924114,0.902459,1.074697,0.870607,0.944230,1.087493,1.036616,...,tensor(0.3053),tensor(-0.9287),tensor(-0.4616),tensor(-0.1855),tensor(0.1151),tensor(-0.8613),tensor(0.7477),tensor(0.3395),tensor(-0.1691),tensor(0.0748)
4,1.083690,0.952222,0.909902,0.814844,0.845148,0.905851,0.912053,0.809988,1.070131,0.993851,...,tensor(0.2623),tensor(-0.9203),tensor(-0.4519),tensor(0.0827),tensor(-0.0945),tensor(-0.7653),tensor(0.7386),tensor(0.4356),tensor(-0.0860),tensor(-0.0833)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
982,0.875689,0.942713,1.061662,0.963320,0.936216,1.092020,1.007864,0.885109,1.234413,0.904729,...,tensor(0.2870),tensor(-0.8791),tensor(-0.4357),tensor(-0.0134),tensor(0.0495),tensor(-0.7856),tensor(0.8730),tensor(0.3396),tensor(-0.2642),tensor(0.0362)
983,0.880198,0.999473,1.111135,0.889674,1.029831,1.014353,1.019814,0.969593,1.051707,1.080274,...,tensor(0.2964),tensor(-1.0083),tensor(-0.4304),tensor(0.0434),tensor(-0.1411),tensor(-0.7327),tensor(0.7527),tensor(0.4104),tensor(-0.0835),tensor(-0.1544)
984,0.919378,0.966359,0.979112,1.018469,0.967341,0.935031,0.967335,0.917793,1.138127,1.061330,...,tensor(0.2496),tensor(-0.9465),tensor(-0.4339),tensor(0.0731),tensor(-0.1033),tensor(-0.7130),tensor(0.7284),tensor(0.3568),tensor(-0.1389),tensor(-0.0972)
985,0.888521,0.871960,1.085654,1.005369,0.912420,0.958169,0.833186,0.892758,0.929759,1.157060,...,tensor(0.1972),tensor(-0.6311),tensor(-0.4613),tensor(0.0107),tensor(-0.0952),tensor(-0.7108),tensor(0.6025),tensor(0.1582),tensor(-0.3760),tensor(-0.1260)


### BERT with Separator 

In [114]:
array_bert_sep = []
for i in df['TxtVecSep']:
    array_bert_sep.append(i) 
df_bert_sep = pd.DataFrame(array_bert_sep)
df_bert_sep


,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
0,tensor(-0.7151),tensor(0.1752),tensor(0.0773),tensor(0.3903),tensor(-0.0146),tensor(0.6560),tensor(-0.2575),tensor(0.0138),tensor(0.2481),tensor(-1.1353),...,tensor(0.2028),tensor(-0.7919),tensor(-0.3987),tensor(0.0346),tensor(-0.0959),tensor(-0.6232),tensor(0.6517),tensor(0.2902),tensor(-0.1253),tensor(-0.0623)
1,tensor(-0.7508),tensor(0.0795),tensor(0.1499),tensor(0.1654),tensor(0.0987),tensor(0.7361),tensor(-0.0802),tensor(0.1004),tensor(0.1747),tensor(-1.1038),...,tensor(0.1337),tensor(-0.7222),tensor(-0.2510),tensor(-0.0317),tensor(-0.1551),tensor(-0.5432),tensor(0.5906),tensor(0.1398),tensor(-0.3562),tensor(-0.1960)
2,tensor(-0.8469),tensor(0.1186),tensor(0.1113),tensor(0.5087),tensor(-0.0769),tensor(0.6186),tensor(-0.2505),tensor(0.0177),tensor(0.3117),tensor(-1.2272),...,tensor(0.2690),tensor(-0.9409),tensor(-0.3813),tensor(0.0567),tensor(-0.0939),tensor(-0.6708),tensor(0.7070),tensor(0.3775),tensor(-0.0416),tensor(-0.1048)
3,tensor(-0.7266),tensor(0.2122),tensor(0.0151),tensor(0.3334),tensor(-0.0200),tensor(0.6780),tensor(-0.3471),tensor(0.0162),tensor(0.3152),tensor(-1.1234),...,tensor(0.2057),tensor(-0.7460),tensor(-0.3752),tensor(-0.0933),tensor(-0.0228),tensor(-0.6693),tensor(0.6397),tensor(0.1956),tensor(-0.2159),tensor(0.1040)
4,tensor(-0.7809),tensor(0.1676),tensor(0.1257),tensor(0.4198),tensor(-0.0726),tensor(0.6679),tensor(-0.2630),tensor(0.0123),tensor(0.2742),tensor(-1.1911),...,tensor(0.2551),tensor(-0.8554),tensor(-0.3988),tensor(0.0763),tensor(-0.1105),tensor(-0.6689),tensor(0.6729),tensor(0.3498),tensor(-0.0761),tensor(-0.0906)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
982,tensor(-0.6961),tensor(0.1646),tensor(0.0735),tensor(0.1886),tensor(-0.0467),tensor(0.7188),tensor(-0.3443),tensor(0.0129),tensor(0.2435),tensor(-1.1424),...,tensor(0.2214),tensor(-0.6981),tensor(-0.3273),tensor(-0.0397),tensor(-0.0426),tensor(-0.5748),tensor(0.7593),tensor(0.2015),tensor(-0.2959),tensor(0.0339)
983,tensor(-0.8358),tensor(0.1242),tensor(0.1159),tensor(0.4736),tensor(-0.0490),tensor(0.6388),tensor(-0.2684),tensor(0.0187),tensor(0.2862),tensor(-1.2119),...,tensor(0.2596),tensor(-0.9033),tensor(-0.3835),tensor(0.0760),tensor(-0.0959),tensor(-0.6717),tensor(0.6693),tensor(0.3656),tensor(-0.0709),tensor(-0.1016)
984,tensor(-0.7940),tensor(0.1501),tensor(0.1051),tensor(0.4256),tensor(-0.0277),tensor(0.6218),tensor(-0.2636),tensor(0.0219),tensor(0.2952),tensor(-1.1679),...,tensor(0.2021),tensor(-0.8570),tensor(-0.3704),tensor(0.0311),tensor(-0.1039),tensor(-0.6248),tensor(0.6649),tensor(0.3159),tensor(-0.1232),tensor(-0.0808)
985,tensor(-0.6608),tensor(0.1827),tensor(0.0520),tensor(0.1995),tensor(0.0154),tensor(0.8367),tensor(-0.2190),tensor(0.0171),tensor(0.2006),tensor(-1.1147),...,tensor(0.2249),tensor(-0.6460),tensor(-0.3840),tensor(0.0198),tensor(-0.0718),tensor(-0.6336),tensor(0.5825),tensor(0.1732),tensor(-0.3096),tensor(-0.0082)


### Resnet & BERT with Separator

In [115]:
frames = [df_resnet, df_bert_sep]
df_bertsep_resnet = pd.concat(frames, axis=1)


df_bertsep_resnet.fillna(0,inplace=True)
df_bertsep_resnet

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
0,0.821871,0.933361,0.920547,0.893624,0.828154,0.966634,0.923863,0.925982,0.881806,1.030577,...,tensor(0.2028),tensor(-0.7919),tensor(-0.3987),tensor(0.0346),tensor(-0.0959),tensor(-0.6232),tensor(0.6517),tensor(0.2902),tensor(-0.1253),tensor(-0.0623)
1,0.973653,0.989243,0.945960,1.021952,0.841379,0.963154,0.870935,0.817570,0.991387,1.050257,...,tensor(0.1337),tensor(-0.7222),tensor(-0.2510),tensor(-0.0317),tensor(-0.1551),tensor(-0.5432),tensor(0.5906),tensor(0.1398),tensor(-0.3562),tensor(-0.1960)
2,1.000431,0.914692,1.031360,0.905025,1.038498,0.961392,0.973464,0.858744,1.051390,1.014303,...,tensor(0.2690),tensor(-0.9409),tensor(-0.3813),tensor(0.0567),tensor(-0.0939),tensor(-0.6708),tensor(0.7070),tensor(0.3775),tensor(-0.0416),tensor(-0.1048)
3,0.876284,0.950117,1.141460,0.924114,0.902459,1.074697,0.870607,0.944230,1.087493,1.036616,...,tensor(0.2057),tensor(-0.7460),tensor(-0.3752),tensor(-0.0933),tensor(-0.0228),tensor(-0.6693),tensor(0.6397),tensor(0.1956),tensor(-0.2159),tensor(0.1040)
4,1.083690,0.952222,0.909902,0.814844,0.845148,0.905851,0.912053,0.809988,1.070131,0.993851,...,tensor(0.2551),tensor(-0.8554),tensor(-0.3988),tensor(0.0763),tensor(-0.1105),tensor(-0.6689),tensor(0.6729),tensor(0.3498),tensor(-0.0761),tensor(-0.0906)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
982,0.875689,0.942713,1.061662,0.963320,0.936216,1.092020,1.007864,0.885109,1.234413,0.904729,...,tensor(0.2214),tensor(-0.6981),tensor(-0.3273),tensor(-0.0397),tensor(-0.0426),tensor(-0.5748),tensor(0.7593),tensor(0.2015),tensor(-0.2959),tensor(0.0339)
983,0.880198,0.999473,1.111135,0.889674,1.029831,1.014353,1.019814,0.969593,1.051707,1.080274,...,tensor(0.2596),tensor(-0.9033),tensor(-0.3835),tensor(0.0760),tensor(-0.0959),tensor(-0.6717),tensor(0.6693),tensor(0.3656),tensor(-0.0709),tensor(-0.1016)
984,0.919378,0.966359,0.979112,1.018469,0.967341,0.935031,0.967335,0.917793,1.138127,1.061330,...,tensor(0.2021),tensor(-0.8570),tensor(-0.3704),tensor(0.0311),tensor(-0.1039),tensor(-0.6248),tensor(0.6649),tensor(0.3159),tensor(-0.1232),tensor(-0.0808)
985,0.888521,0.871960,1.085654,1.005369,0.912420,0.958169,0.833186,0.892758,0.929759,1.157060,...,tensor(0.2249),tensor(-0.6460),tensor(-0.3840),tensor(0.0198),tensor(-0.0718),tensor(-0.6336),tensor(0.5825),tensor(0.1732),tensor(-0.3096),tensor(-0.0082)


## 6)++ BERT with Separator & Resnet34

In [116]:
target_idx = 265677

In [117]:
X = df_bertsep_resnet
model = NearestNeighbors(n_neighbors=5,
                         metric='cosine',
                         algorithm='brute')
model.fit(X) 

NearestNeighbors(algorithm='brute', metric='cosine')

In [118]:
from random import randrange
query_index = df[df['Unnamed: 0.1']==target_idx].index[0] #new
query_high_level = df.iloc[query_index]['high_level']
high_level_indices = df[df['high_level'] == query_high_level].index
# print(df.iloc[high_level_indices.to_list()]['high_level'])
print("Input product index is", query_index)
distances,indices= model.kneighbors(X.iloc[query_index,:].values.reshape(1,-1))
# print(indices)

Input product index is 114


In [119]:
from PIL import Image
import requests
from io import BytesIO

data = df
img = [data['image_links'][query_index][0]]
for i in range(1,len(distances.flatten())):
    url = data['image_links'].iloc[indices.flatten()[i]][0]
    img.append(url)

from IPython.display import HTML, display, Markdown, Latex

for i in range(len(img)):
    #display(Markdown("![Item]("+img[i]+")"))
    if i!=0:
        print('This is recommended item', i)
    else:
        print('This is input item')
    display(Markdown("<img src=\""+img[i]+"\"  width='40%' height='30%'>"))

This is input item


<img src="https://di2ponv0v5otw.cloudfront.net/posts/2022/07/23/62dca3e6bb5937efc1b4b443/s_62dca3fbbcbb522134728bf4.jpg"  width='40%' height='30%'>

This is recommended item 1


<img src="https://di2ponv0v5otw.cloudfront.net/posts/2022/04/04/624b4bf667bd91d9aa33194c/s_624b4c1a941f17c46f578e75.jpg"  width='40%' height='30%'>

This is recommended item 2


<img src="https://di2ponv0v5otw.cloudfront.net/posts/2022/03/07/62264bde6f6c91a04905ad9b/s_62264c0a3a0db968c05479f4.jpg"  width='40%' height='30%'>

This is recommended item 3


<img src="https://di2ponv0v5otw.cloudfront.net/posts/2022/08/13/62f895ce97b5d083d1e669cc/s_62f8960217fb4bfe0d5fafaf.jpg"  width='40%' height='30%'>

This is recommended item 4


<img src="https://di2ponv0v5otw.cloudfront.net/posts/2022/08/02/62e96811a0aeb7170756cee1/s_62e9682c17e49c5e64bf3133.jpg"  width='40%' height='30%'>

In [120]:
df_bert_resnet_same_level = pd.DataFrame(df_bert_resnet.iloc[high_level_indices.to_list()])
model_new = NearestNeighbors(n_neighbors=5,
                         metric='cosine',
                         algorithm='brute')
X=df_bert_resnet_same_level.dropna()
# print(len(X))
# print(X)
model_new.fit(X) 
distances_new,indices_new= model_new.kneighbors(X.iloc[query_index,:].values.reshape(1,-1))
from PIL import Image
import requests
from io import BytesIO
# print(indices_new)

data = pd.DataFrame(df.iloc[high_level_indices.to_list()])
img = [data['image_links'][query_index][0]]
for i in range(1,len(distances_new.flatten())):
    url = data['image_links'].iloc[indices_new.flatten()[i]][0]
    img.append(url)

from IPython.display import HTML, display, Markdown, Latex

for i in range(len(img)):
    #display(Markdown("![Item]("+img[i]+")"))
    if i!=0:
        print('This is recommended item', i, 'with index')
    else:
        print('This is input item')
    display(Markdown("<img src=\""+img[i]+"\"  width='40%' height='30%'>"))

This is input item


<img src="https://di2ponv0v5otw.cloudfront.net/posts/2022/07/23/62dca3e6bb5937efc1b4b443/s_62dca3fbbcbb522134728bf4.jpg"  width='40%' height='30%'>

This is recommended item 1 with index


<img src="https://di2ponv0v5otw.cloudfront.net/posts/2022/08/08/62f106b880a5d6559494d6c3/s_62f106b880a5d6559494d6c4.jpeg"  width='40%' height='30%'>

This is recommended item 2 with index


<img src="https://di2ponv0v5otw.cloudfront.net/posts/2022/02/02/61fb10a6c32ee6cf336cd3ab/s_61fb10c1800f647921bbaeb9.jpg"  width='40%' height='30%'>

This is recommended item 3 with index


<img src="https://di2ponv0v5otw.cloudfront.net/posts/2022/06/22/62b370daff7c5a8da1ab0d01/s_62b482e6008b99d56df48dce.jpg"  width='40%' height='30%'>

This is recommended item 4 with index


<img src="https://di2ponv0v5otw.cloudfront.net/posts/2022/07/28/62e3339dbd66cd5a884ae163/s_62e333af92e49131088a61e1.jpg"  width='40%' height='30%'>